<a href="https://colab.research.google.com/github/aayushs879/Soc-Gen-Complaint-Status-Tracking/blob/master/soc_gen_complaint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving languages.csv to languages.csv
Saving test.csv to test.csv
Saving train.csv to train.csv
User uploaded file "languages.csv" with length 129798 bytes
User uploaded file "test.csv" with length 22223361 bytes
User uploaded file "train.csv" with length 53428139 bytes


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split as tts
from xgboost import XGBClassifier
import nltk

In [0]:
train = pd.read_csv('train.csv')


eng = {}
fr = {}
es = {}

languages = pd.read_csv('languages.csv', header = None)
languages.columns = ['one']



text = train['Consumer-complaint-summary']

for i in range(len(languages)):
    if languages.one[i] == 'fr':
        fr.update({i: text[i]})
    elif languages.one[i] == 'es':
        es.update({i: text[i]})
    elif languages.one[i] == 'en':
        eng.update({i:text[i]})


In [0]:

from nltk.corpus import stopwords
#nltk.download('stopwords')

swen = set(stopwords.words('english'))
swfr = set(stopwords.words('french'))
swes = set(stopwords.words('spanish'))


from nltk.stem.porter import PorterStemmer

frcor = {}
for key in fr:
    words = fr[key]
    words = words.lower()
    ps = PorterStemmer()
    words = words.split()
    sentence = [ps.stem(word) for word in words if not word in swfr]
    frcor.update({key:' '.join(sentence)})

encor = {}
for key in eng:
    words = eng[key]
    words = words.lower()
    ps = PorterStemmer()
    words = words.split()
    sentence = [ps.stem(word) for word in words if not word in swen]
    encor.update({key:' '.join(sentence)})


escor = {}
for key in es:
    words = es[key]
    words = words.lower()
    ps = PorterStemmer()
    words = words.split()
    sentence = [ps.stem(word) for word in words if not word in swes]
    escor.update({key:' '.join(sentence)})

In [0]:
from sklearn.feature_extraction.text import HashingVectorizer

hven = HashingVectorizer(n_features = 3000)
hvfr = HashingVectorizer(n_features = 2000)
hves = HashingVectorizer(n_features = 2000)


Xen = hven.fit_transform(encor.values()).toarray()
Xfr = hvfr.fit_transform(frcor.values()).toarray()
Xes = hves.fit_transform(escor.values()).toarray()


In [0]:
df = pd.DataFrame(index = range(len(escor) + len(encor) + len(frcor)), columns = ['delayed','language',  'Complaint-status', 'Consumer-Dispute', 'Complaint-reason'])


c = []
for i in range(len(train)):
    a = train['Date-sent-to-company'][i].split('/')
    b = train['Date-received'][i].split('/')
    c.append(int(a[2])*365 + int(a[1])*30 + int(a[0]) - int(b[2])*365 -int(b[1])*30 -int(b[0]))


train['delayed'] = c

i = 0
for key in encor:
    df['delayed'][i] = train['delayed'][int(key)]
    df['Complaint-status'][i] = train['Complaint-Status'][int(key)]
    df['Consumer-Dispute'][i] = train['Consumer-disputes'][int(key)]
    df['Complaint-reason'][i] = train['Complaint-reason'][int(key)]
    df['language'][i] = languages['one'][int(key)]
    i+=1



for key in frcor:
    df['delayed'][i] = train['delayed'][int(key)]
    df['Complaint-status'][i] = train['Complaint-Status'][int(key)]
    df['Consumer-Dispute'][i] = train['Consumer-disputes'][int(key)]
    df['Complaint-reason'][i] = train['Complaint-reason'][int(key)]
    df['language'][i] = languages['one'][int(key)]
    i+=1

for key in escor:
    df['delayed'][i] = train['delayed'][int(key)]
    df['Complaint-status'][i] = train['Complaint-Status'][int(key)]
    df['Consumer-Dispute'][i] = train['Consumer-disputes'][int(key)]
    df['Complaint-reason'][i] = train['Complaint-reason'][int(key)]
    df['language'][i] = languages['one'][int(key)]
    i+=1


In [0]:
df['Consumer-Dispute'] = df['Consumer-Dispute'].fillna('not known')



y = df['Complaint-status'].iloc[:].values


from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

df = df.drop(['Complaint-status'], axis = 1)

X1 = df.apply(le.fit_transform)

X1 = X1.iloc[:, :].values


In [0]:
pdxen = np.zeros(shape = (Xen.shape[0], 4000))
pdfr_prev = np.zeros(shape = (Xfr.shape[0], 3000))
pdfr_post = np.zeros(shape = (Xfr.shape[0], 2000))
pdes = np.zeros(shape = (Xes.shape[0], 5000))


Xen = np.concatenate((Xen, pdxen), axis = 1)
Xfr = np.concatenate((pdfr_prev, Xfr, pdfr_post), axis = 1)
Xes = np.concatenate((pdes, Xes), axis = 1)

In [0]:
del encor
del frcor
del eng
del escor
del fr
del es
del sentence
del train
del swen
del swfr
del swes
del a
del b
del c
del df
del i


In [0]:
del languages

In [0]:
X2 = np.concatenate((Xen, Xfr, Xes), axis = 0)
X = np.concatenate((X1, X2), axis = 1)

X_train, X_test, y_train, y_test = tts(X, y, test_size = 4500)

In [0]:
del Xen
del Xfr
del Xes
del X
del y

In [0]:
model = XGBClassifier()
model.fit(X_train, y_train)
del X_train
del y_train

In [0]:
y_pred = model.predict(X_test)
from sklearn.metrics import f1_score
fs = f1_score(y_test, y_pred, average = 'weighted')

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
print('The weighted f1 score is ',fs)

The weighted f1 score is  0.7102678005698054


In [0]:
import pickle
pickle.dump(model, open('soc-gen_complaint.model', 'wb'))

In [0]:
from google.colab import files

files.download('soc-gen_complaint.model')

In [0]:
m1 = pickle.load(open('soc-gen_complaint.model', 'rb'))

In [0]:
(m1.predict(X_test) == 'Closed with explanation').sum()/len(y_test)

0.988

In [0]:
test = pd.read_csv('test.csv')

In [0]:
c = []
for i in range(len(test)):
    a = test['Date-sent-to-company'][i].split('/')
    b = test['Date-received'][i].split('/')
    c.append(int(a[2])*365 + int(a[1])*30 + int(a[0]) - int(b[2])*365 -int(b[1])*30 -int(b[0]))


In [11]:
from google.colab import drive
drive.mount('/content/gdrive')

import os 
path = os.path.abspath('/content/gdrive/My Drive/HackerEarth/Soc-Gen')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [13]:
#!pip install langdetect

from langdetect import DetectorFactory
from langdetect import detect
from langdetect import detect_langs 
DetectorFactory.seed = 0

    100% |████████████████████████████████| 1.0MB 19.4MB/s 
  Running setup.py bdist_wheel for langdetect ... - \ | done
  Stored in directory: /root/.cache/pip/wheels/ec/0c/a9/1647275e7ef5014e7b83ff30105180e332867d65e7617ddafe
Successfully built langdetect


In [14]:
test = pd.read_csv(os.path.join(path, 'test.csv'))
print(test.columns)
print(test.head())
print(test.nunique())
print(test.isnull().sum())

Index(['Complaint-ID', 'Date-received', 'Transaction-Type', 'Complaint-reason',
       'Company-response', 'Date-sent-to-company', 'Consumer-disputes',
       'Consumer-complaint-summary'],
      dtype='object')
  Complaint-ID Date-received         Transaction-Type  \
0         Te-1     8/18/2016  Bank account or service   
1         Te-2     4/18/2016          Debt collection   
2         Te-3     3/23/2016         Credit reporting   
3         Te-4     6/26/2017             Student loan   
4         Te-5     5/13/2016         Credit reporting   

                          Complaint-reason  \
0  Account opening, closing, or management   
1                    Communication tactics   
2   Incorrect information on credit report   
3     Dealing with your lender or servicer   
4   Incorrect information on credit report   

                                    Company-response Date-sent-to-company  \
0  Company has responded to the consumer and the ...            8/18/2016   
1  Company bel

In [0]:
languages = []
for i in range(len(test)):
  languages.append(detect(test['Consumer-complaint-summary'][i]))

In [19]:
languages = pd.DataFrame(languages, columns = ['one'])
print(languages.nunique())

one    15
dtype: int64
